In [ ]:
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
train = pd.read_csv("/content/Clean_train.csv")
test = pd.read_csv("/content/Clean_test.csv")

In [ ]:
x = train['text']
y = train['target']
xx_test = test["text"]

In [ ]:
xx_test.isnull().sum()

1

In [ ]:
xx_test.fillna('', inplace=True)

In [ ]:
xx_test.isnull().sum()

0

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
x_tfidf = tfidf_vectorizer.fit_transform(x)
x_test_tfidf = tfidf_vectorizer.transform(xx_test)

In [ ]:
nb_model = MultinomialNB()
lr_model = LogisticRegression()
knn_model = KNeighborsClassifier(n_neighbors=5)

In [ ]:
stacking_model = StackingClassifier(
    estimators=[('nb', nb_model), ('lr', lr_model), ('knn', knn_model)],
    final_estimator=LogisticRegression(),
    stack_method='predict_proba',
    cv=KFold(n_splits=5, shuffle=True, random_state=42)
)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_tfidf,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42 ,
                                                    shuffle = True)

In [ ]:
stacking_model.fit(x_train, y_train)

StackingClassifier(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimators=[('nb', MultinomialNB()),
                               ('lr', LogisticRegression()),
                               ('knn', KNeighborsClassifier())],
                   final_estimator=LogisticRegression(),
                   stack_method='predict_proba')

In [ ]:
y_pred_stacking = stacking_model.predict(x_test)

In [ ]:
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)
print("Accuracy of Stacking Classifier with 5-fold cross-validation:", accuracy_stacking)


Accuracy of Stacking Classifier with 5-fold cross-validation: 0.7977675640183848


In [ ]:
y_pred_stacking = stacking_model.predict(x_test_tfidf)

submission6 = pd.DataFrame({
    "id": test["id"],
    "target": y_pred_stacking
})

In [ ]:
submission6.to_csv("Submission6.csv", index=False)